## Import Libraries

In [1]:
import datetime as dt

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn import preprocessing

import pyarrow.parquet as pq
import s3fs

import pandas as pd
tqdm.pandas()

## Start with the STB DataSet

In [2]:
day = "2018-07-01"

In [3]:
s3 = s3fs.S3FileSystem()
data = pq.ParquetDataset("s3://columbia-stb/local_day_id="+day+"/media_platform_type_cd=D", 
                         filesystem=s3).read_pandas().to_pandas()


In [4]:
to_drop = ['time_zone', 'unique_session_id', 'program_viewing_duration_seconds',
           'station_language', 'start_timestamp_utc', 'end_timestamp_utc',
           'viewing_end_timestamp_utc', 'viewing_end_timestamp_local',
           'station_timezone','channel_language','company_description',
          'listing_id','listing_starttime', 'listing_endtime', 
           'listing_airingtype','program_id','program_title', 
           'program_shortsynopsis', 'program_mediumsynopsis', 'program_longsynopsis',
          'program_adult', 'program_local','program_sportssubtitle',
          'adm_zip','adm_customer_type','vodasset_provider_id', 'station_title', 'start_timestamp_local', 'end_timestamp_local', 'program_contentrating']
data.drop(columns = to_drop, inplace = True)

In [5]:
data["viewing_duration_seconds"] = data["viewing_duration_seconds"].progress_apply(lambda x: float(x))
data["duration_seconds"] = data["duration_seconds"].progress_apply(lambda x: float(x))

### Remove the rows with less than 6 minutes viewing duration

In [6]:
data = data[data["viewing_duration_seconds"] >= 360]
data.head()

,di_account_num,duration_seconds,viewing_duration_seconds,station_tagids_title_type,company_displayname,program_runtime,program_tagids_title_type
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,618.0,618.0,"Station:SDTV,Station:Digital",TNT,60.0,"Genre:Crime drama,KidsTheme:older teens (ages ..."
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,1753.0,1753.0,"Station:Sports,Station:TVE Rating: TVY,Station...",ESPN,115.0,"Genre:SportingEvent,ITheme:Motor Sports,Genre:..."
5,MHraetBu6LHZshYarbQc1C9kQ9RNd5bYgvApEt0kHdc=,1331.0,1331.0,"Station:Digital,Station:HDTV,Station:TVE Ratin...",OWN,60.0,"KidsTheme:teens (ages 13-14),Genre:Drama,IThem..."
9,L+9/QpVCfQBVfmpXAHdnsy03VvFhNnqh9mRupjGb0+8=,400.0,400.0,"Station:SDTV,Station:Digital",HGTV,60.0,"Genre:Drama,Genre:Home improvement,ITheme:Dram..."
11,tBpyUyVDRxBElFljI5WqWxEx8/3hpQUPk1U8lXLOzaM=,1103.0,1103.0,"Station:Sports,Station:Digital,Station:SDTV",ESPN,NaN,"ITheme:Sports,Genre:Auto Racing,Genre:Sporting..."


### Create dummy columns for type of stations

In [7]:
def create_stationid(x):
    """
    This function will create 6 dummy columns containing the type of stations used to watch TV.
    These 6 columns are (in order): Digital, Analago, HDTV, SDTV, Sports, TVE
    """
    values = [0,0,0,0]
    
    if x.find('HDTV')>-1:
        values[0] = 1
    if x.find('SDTV')>-1:
        values[1] = 1
    if x.find('Sports')>-1:
        values[2] = 1
    if x.find('TVE')>-1:
        values[3] = 1
    return values

data[['station_hdtv','station_sdtv','station_sports','station_tve']] = data.progress_apply(lambda x: create_stationid(x['station_tagids_title_type']), axis = 1, result_type = 'expand')

100%|██████████| 4476687/4476687 [05:17<00:00, 14108.69it/s]


In [8]:
data.drop(columns = ["station_tagids_title_type"], inplace = True)

### Create dummy columns Networks (TV Channels)

In [9]:
data = pd.concat([data, pd.get_dummies(data["company_displayname"])], axis = 1)

In [10]:
data.head()

,di_account_num,duration_seconds,viewing_duration_seconds,company_displayname,program_runtime,program_tagids_title_type,station_hdtv,station_sdtv,station_sports,station_tve,...,OWN,SYFY,TBS,TLC,TNT,UNIVERSO,UP,USA,VH1,VICELAND
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,618.0,618.0,TNT,60.0,"Genre:Crime drama,KidsTheme:older teens (ages ...",0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,1753.0,1753.0,ESPN,115.0,"Genre:SportingEvent,ITheme:Motor Sports,Genre:...",1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,MHraetBu6LHZshYarbQc1C9kQ9RNd5bYgvApEt0kHdc=,1331.0,1331.0,OWN,60.0,"KidsTheme:teens (ages 13-14),Genre:Drama,IThem...",1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
9,L+9/QpVCfQBVfmpXAHdnsy03VvFhNnqh9mRupjGb0+8=,400.0,400.0,HGTV,60.0,"Genre:Drama,Genre:Home improvement,ITheme:Dram...",0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
11,tBpyUyVDRxBElFljI5WqWxEx8/3hpQUPk1U8lXLOzaM=,1103.0,1103.0,ESPN,NaN,"ITheme:Sports,Genre:Auto Racing,Genre:Sporting...",0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


### Examine Program_tagids_title_type

In [11]:
def get_genre(text):
    try:
        start = text.find('Genre:') + 6
        text = text[start:]
        text = text.split(',')[0]
    except:
        pass
    return text

data['genre'] = data['program_tagids_title_type'].progress_apply(lambda x: get_genre(x))

100%|██████████| 4476687/4476687 [00:07<00:00, 619332.74it/s]


In [12]:
genre_tab = pd.read_csv('/home/ubuntu/Notebooks/Genre Dictionary corrected.csv', sep = ';')

genre_col = genre_tab.columns
col = 0
genre_dict = {}
# Drama, Sports, Reality, Entertainment, Documentary, Kids, Family, Animals, News, Movie, Music, Teen, Science & Technology, Health & Fitness, Religion, Shopping, Misellaneous

all_genre = []

for j in range(len(genre_col)):
    for i in genre_tab[genre_col[j]].dropna().unique():
        if i not in all_genre:
            all_genre.append(i)
            genre_dict[i] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


for j in range(len(genre_col)):
    for i in genre_tab[genre_col[j]].dropna().unique():
        genre_dict[i][j] = 1


genre_tab.head()

,Drama,Sports,Reality,Entertainment,Documentary,Kids,Family,Animals,News,Movie,Music,Teen,Science & Technology,Health & Fitness,Religion,Shopping,Miscellaneous
0,Action & Adventure,Sports talk,Legal,Action & Adventure,Nature,Cartoon & Animation,Children's/Family Entertainment,Pets,Nature,Thriller,Western,Comedy,Sci-Fi & Fantasy,Health,Fundraiser/Telethon,Consumer,Anthology
1,Fantasy,Basketball,Outdoors,Fantasy,Crime,Animated,Home & How-To,Wild animals,Real Estate,Military & War,Local,Education,Aviation,Fitness,Religion,Fashion,Special
2,Miniseries,Outdoors,Talk,Martial Arts,History & Biography,Comedy,Home & Garden,Ocean creatures,Crime,Legal,Musical,Romance,Environment,Health & Fitness,NaN,Thanksgiving,Independent
3,Crime,Martial Arts,Travel,Travel,Medical,Education,Animated,Horses and farm animals,Medical,Western,Hip-Hop & Rap Music,Teens,Science and nature,NaN,NaN,Valentine's Day,Variety
4,Thriller,Tennis,Food,Sitcom,Military & War,Holiday,Medical,Animals,Newscast,Horror,Musical,Mystery,Robots,NaN,NaN,Halloween,How-To


In [13]:
def get_genre_col(genre):
    try:
        return np.array(genre_dict[genre])
    except:
        return np.array([np.nan]*17)

data[['Drama_view', 'Sports_view', 'Reality_view', 'Entertainment_view', 'Documentary_view', 'Kids_view', 'Family_view', 'Animals_view', 'News_view', 'Movie_view', 'Music_view', 'Teen_view', 'Science & Technology_view', 'Health & Fitness_view', 'Religion_view', 'Shopping_view', 'Misellaneous_view']] = data.progress_apply(lambda x: x["viewing_duration_seconds"]*get_genre_col(x['genre']), axis = 1, result_type = 'expand')
data[['Drama_d', 'Sports_d', 'Reality_d', 'Entertainment_d', 'Documentary_d', 'Kids_d', 'Family_d', 'Animals_d', 'News_d', 'Movie_d', 'Music_d', 'Teen_d', 'Science & Technology_d', 'Health & Fitness_d', 'Religion_d', 'Shopping_d', 'Misellaneous_d']] = data.progress_apply(lambda x: x["duration_seconds"]*get_genre_col(x['genre']), axis = 1, result_type = 'expand')




100%|██████████| 4476687/4476687 [05:17<00:00, 14091.17it/s]


In [14]:
data.drop(columns = ["program_tagids_title_type"], inplace = True)

In [15]:
data.drop(columns = ["program_runtime"], inplace = True)

In [16]:
data.head()

,di_account_num,duration_seconds,viewing_duration_seconds,company_displayname,station_hdtv,station_sdtv,station_sports,station_tve,A&E,AMC,...,Animals_d,News_d,Movie_d,Music_d,Teen_d,Science & Technology_d,Health & Fitness_d,Religion_d,Shopping_d,Misellaneous_d
1,tPUCiqLU341PYO17j/iJjVj15wJ5996DGOZt5hBuxPk=,618.0,618.0,TNT,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,gxlzvOuWeB0UIR/j9oO/kNpExoqCtWrelTATAvcnGFE=,1753.0,1753.0,ESPN,1,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,MHraetBu6LHZshYarbQc1C9kQ9RNd5bYgvApEt0kHdc=,1331.0,1331.0,OWN,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,L+9/QpVCfQBVfmpXAHdnsy03VvFhNnqh9mRupjGb0+8=,400.0,400.0,HGTV,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,tBpyUyVDRxBElFljI5WqWxEx8/3hpQUPk1U8lXLOzaM=,1103.0,1103.0,ESPN,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
grouped = data.groupby("di_account_num").sum()
grouped.head()

,duration_seconds,viewing_duration_seconds,station_hdtv,station_sdtv,station_sports,station_tve,A&E,AMC,BET,CMT,...,Animals_d,News_d,Movie_d,Music_d,Teen_d,Science & Technology_d,Health & Fitness_d,Religion_d,Shopping_d,Misellaneous_d
di_account_num,,,,,,,,,,,,,,,,,,,,,
++++yXWkYNIEvfB83VGZTvx8hyDTT0WthqMuggmF9Iw=,2322.0,2322.0,4,0,0,0,4,0,0,0,...,0.0,2322.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+++5Cs2yHlqypQ0OybE3Ga0Hj5Fuf2ZCA6uCazOxFk8=,580.0,580.0,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,580.0
+++Brw+WK4d8VrHsTFBTZaXFBGFn4Zy/ZmimCXjSOjw=,844.0,844.0,1,0,0,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+++HgCK7ZJ1M+Y4rl8wz7GziaC1qvZRcEjqv9BaHSPk=,3716.0,3716.0,2,0,0,2,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+++NblXovoqDgo/sturhLyeDExOX5VCU0kDqDdae8BE=,3661.0,1800.0,1,0,0,1,0,0,0,0,...,0.0,3661.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
grouped.to_csv("/home/ubuntu/Notebooks/Data/Preprocessed_data_"+day+".csv", index = True)

In [66]:
#
numerator_columns = list(grouped.columns)[39:56]
denominator_columns = list(grouped.columns)[56:]

ratios = pd.DataFrame(np.array(grouped.loc[:,numerator_columns])/ np.array(grouped.loc[:,denominator_columns]), columns = ['Drama_ratio', 'Sports_ratio', 'Reality_ratio', 'Entertainment_ratio', 'Documentary_ratio', 'Kids_ratio', 'Family_ratio', 'Animals_ratio', 'News_ratio', 'Movie_ratio', 'Music_ratio', 'Teen_ratio', 'Science & Technology_ratio', 'Health & Fitness_ratio', 'Religion_ratio', 'Shopping_ratio', 'Misellaneous_ratio'])    

grouped = pd.concat([grouped, ratios], axis = 1)
grouped.fillna(0, inplace = True)
grouped.head()


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


,duration_seconds,viewing_duration_seconds,station_hdtv,station_sdtv,station_sports,station_tve,A&E,AMC,BET,CMT,...,Animals_ratio,News_ratio,Movie_ratio,Music_ratio,Teen_ratio,Science & Technology_ratio,Health & Fitness_ratio,Religion_ratio,Shopping_ratio,Misellaneous_ratio
++++yXWkYNIEvfB83VGZTvx8hyDTT0WthqMuggmF9Iw=,2322.0,2322.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+++5Cs2yHlqypQ0OybE3Ga0Hj5Fuf2ZCA6uCazOxFk8=,580.0,580.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+++Brw+WK4d8VrHsTFBTZaXFBGFn4Zy/ZmimCXjSOjw=,844.0,844.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+++HgCK7ZJ1M+Y4rl8wz7GziaC1qvZRcEjqv9BaHSPk=,3716.0,3716.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+++NblXovoqDgo/sturhLyeDExOX5VCU0kDqDdae8BE=,3661.0,1800.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
grouped.to_csv("")

['Drama_ratio', 'Sports_ratio', 'Reality_ratio', 'Entertainment_ratio', 'Documentary_ratio', 'Kids_ratio', 'Family_ratio', 'Animals_ratio', 'News_ratio', 'Movie_ratio', 'Music_ratio', 'Teen_ratio', 'Science & Technology_ratio', 'Health & Fitness_ratio', 'Religion_ratio', 'Shopping_ratio', 'Misellaneous_ratio']
